# Classifying Taxonomy and MAG abundances

Using GTDB-Tk: https://github.com/Ecogenomics/GTDBTk?tab=readme-ov-file GTDB-Tk identifies a core set of genes in the MAG, and use these to compute a species phylogeny

**The gtdb reference database needs 100GB space so do not create conda env in /home directory. Either create the env in /work or /scratch**

I used this opportunity to create a /scratch directory with this documentation: https://docs.unity.rc.umass.edu/documentation/managing-files/hpc-workspace/

In [ ]:
#create scratch directory
ws_allocate gtdb 30

#FOR LATER:to release scratch space when you don't need it anymore
ws_release gtdb

In [ ]:
Info: creating workspace.
/scratch3/workspace/nikea_ulrich_uml_edu-gtdb
remaining extensions  : 3
remaining time in days: 30

In [ ]:
#INSTALLATION of gtdbtk in /scratch
cd /scratch3/workspace/nikea_ulrich_uml_edu-gtdb
module load conda/latest
mkdir -p /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs
conda create --prefix /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy python=3.8
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy
conda install -c conda-forge -c bioconda gtdbtk=2.4.0

In [ ]:
#conda package is bundled with a script that will automatically download, and extract the GTDB-Tk reference data, simply run:
download-db.sh
#This will take awhile -- massive database

https://ecogenomics.github.io/GTDBTk/commands/classify_wf.html

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=150G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 48:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/mcav/slurm-gtdb_taxonomy-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

# Set parameters
SAMPLENAME="mcav"
BINPATH="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/binning/${SAMPLENAME}/${SAMPLENAME}_DASTool_bins"
OUTDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/${SAMPLENAME}"
mkdir -p $OUTDIR

#Run gtdb-tk
gtdbtk classify_wf -x fa --skip_ani_screen \
--genome_dir $BINPATH/ --out_dir $OUTDIR/gtdb_out

#This will process all genomes in the directory <my_genomes> using both bacterial and archaeal marker sets and place the results in <output_dir>.
#Genomes must be in FASTA format (gzip with the extension .gz is acceptable)

# JOB-ID:
# bash script file name: 

To be able to look at trees from gtdb-tk (which uses FastTree) in itol, use this script. However better practice is to take the protein alignments and construct trees with IQTree.

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=100G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/mcav/slurm-itol-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

SAMPLENAME="mcav"
WORKDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/${SAMPLENAME}/gtdb_out/classify"
OUTDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/${SAMPLENAME}/gtbd_trees_for_vis"
mkdir -p $OUTDIR

#convert all classify trees to itol readable format
gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.bac120.classify.tree.4.tree --output $OUTDIR/itol.gtdbtk.classify.tree.4.tree

gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.bac120.classify.tree.6.tree --output $OUTDIR/itol.gtdbtk.classify.tree.6.tree

gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.bac120.classify.tree.8.tree --output $OUTDIR/itol.gtdbtk.classify.tree.8.tree

gtdbtk convert_to_itol --input $WORKDIR/gtdbtk.backbone.bac120.classify.tree --output $OUTDIR/itol.gtdbtk.backbone.classify.tree

# JOB-ID:
# bash script file name: 

## IQ-Tree
Infer a phylogeny with the multiple sequence alignment from gtdbtk output

http://www.iqtree.org/doc/

In [ ]:
#INSTALLATION
module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy
conda install -c bioconda iqtree

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=100G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/slurm-iqtree-%j.out  # %j = job ID  # %j = job ID

module load conda/latest
conda activate /scratch3/workspace/nikea_ulrich_uml_edu-gtdb/envs/taxonomy

#set parameters
MSAPATH='/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/gtdb_allbins_out/align'
MSA="gtdbtk.bac120.user_msa.fasta.gz"
OUTDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/taxonomy/iqtree"
mkdir -p $OUTDIR

cd $OUTDIR

#run iqtree
iqtree -s $MSAPATH/$MSA -T AUTO -m TEST -alrt 1000 -B 1000 --prefix all_bins

# JOB-ID: 
# bash script file name: 

## CoverM 
https://wwood.github.io/CoverM/coverm-genome.html

Map the reads of each sample back to the MAG catalogue and retrieve mapping statistics. This gives you relative abundance information to measure how abundant or rare each bacteria was in each sample.

In [ ]:
#Install CoverM in the anvi'o environment 
module load conda/latest
conda activate anvio-8
conda install -c bioconda coverm

**note: added bin identifier to each contig in MAG assembly files so that when they are concatenated into 1 MAG database, the MAGs can be identified still**


In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=180G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 24:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/annotation/dlab/slurm-MAG-mapping-%j.out  # %j = job ID

module load conda/latest
conda activate anvio-8


SAMPLENAME="dlab"
READSPATH="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/assembly/${SAMPLENAME}/repaired"
MAGPATH="/scratch3/workspace/nikea_ulrich_uml_edu-gtdb/bins/${SAMPLENAME}"
WORKDIR="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/annotation/${SAMPLENAME}"
mkdir -p "$WORKDIR"
MAGDB="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/annotation/${SAMPLENAME}/MAG_db"
mkdir -p "$MAGDB"
XTRAFILES="/scratch3/workspace/nikea_ulrich_uml_edu-gtdb/annotation/${SAMPLENAME}"
mkdir -p "$XTRAFILES"
LISTPATH="/work/pi_sarah_gignouxwolfsohn_uml_edu/nikea/COL/"
SAMPLELIST="032024_${SAMPLENAME}_sampleids.txt"


#concatenate MAGs into 1 file to create a MAG database 
#make sure MAG ID (bin#) has been addded to the beginning of each contig so that you can tell the bins apart
cat $MAGPATH/*.fa > $MAGDB/"$SAMPLENAME"_mags.fsa

cd $MAGDB
#index MAG database with bowtie2
bowtie2-build --large-index --threads 11 "$SAMPLENAME"_mags.fsa "$SAMPLENAME"_index

while IFS= read -r SAMPLEID; do
    #align reads to your contigs and collects that in a .sam file
    bowtie2 --threads 11 -x "$SAMPLENAME"_index -1 $READSPATH/"${SAMPLEID}"_host_removed_R1.tagged_filter_ready.fastq.gz -2 $READSPATH/"${SAMPLEID}"_host_removed_R2.tagged_filter_ready.fastq.gz -S $XTRAFILES/"${SAMPLEID}".sam
    #make sure to point it to the index not the FIXEDCON file (-x parameter)
    
    #converts your sam file to a bam file, but its neither sorted nor indexed, so we use an Anvi'O script to do so:
    samtools view -F 4 -b -S $XTRAFILES/"${SAMPLEID}".sam -o $WORKDIR/"${SAMPLEID}"-RAW.bam
   
    #index and sort your bam file
    anvi-init-bam $WORKDIR/"${SAMPLEID}"-RAW.bam -o $WORKDIR/"${SAMPLEID}".bam
    
    rm $WORKDIR/"${SAMPLEID}"-RAW.bam
done < "$LISTPATH/${SAMPLELIST}"
echo "Mapping success!"

#extract stats with CoverM
#relative abundance
coverm genome \
    -b $WORKDIR/*.bam \
    -s . \
    -m relative_abundance \
    --min-covered-fraction 0 \
    -o $WORKDIR/mapping_rate

#MAG coverage
coverm genome \
    -b $WORKDIR/*.bam \
    -s . \
    -m count covered_fraction \
    --min-covered-fraction 0 \
    -o $WORKDIR/count_table

#-s flag is the character (".") that separates the MAG ID from the rest of the contig info.  

conda deactivate

#JOB ID:
#bash script:

use EnrichM for better taxonomic profiling of MAGs: https://github.com/geronimp/enrichM?tab=readme-ov-file ?